# Gender inequality across OECD-countries
In this assignment, we conduct a descriptive analysis of the development in the gender gap across a selection of OECD-countries. In addition, we investigate potential causes of differences in the gender gap by plotting the development in net child care costs, access to formal child care and length of paternity leave. The goal is to gain deeper understanding of the potential drivers of the norm-effect that we incorperated in our simple model of household behavior in assignment one. 

All data has been downloaded from the OECD-data bank (https://stats.oecd.org/) and uploaded to our repository.

# Code struture

a) Import packages and load data. 

b) Gender gap.

c) Net childcare costs. 

d) Parental leave and formal child care. 


# a. Import packages and load data

In [12]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#from matplotlib_venn import venn2

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Load data
df_gg = pd.read_csv('Gender gap.csv')
df_pl = pd.read_csv('Parental leave.csv')
df_f = pd.read_csv('Family.csv')
df_nc = pd.read_csv('netchildcare costs.csv')

# b. Gender gap
1) We read the OECD-data set for the gender gap across countries. We drop irrelevant columns and rename 
2) We clean the data-set 
3) We make an interactive plot that illustrates the development in the gender gap over time for a selection of OECD-countries
4) We create a data set with only 2020-observations for later use


In [14]:
# Showing the gender gap dataset by a pandas DataFrame
#dt_gender_gap = pd.DataFrame(df_gg)
#dt_true.head(5)


In [15]:

# Drop collumns that we do not need
drop_these = ['COUNTRY','SEX','Sex','Series','TIME','Unit','Unit Code','PowerCode Code','PowerCode','Reference Period Code','Reference Period','Flag Codes','Flags']
df_gg.drop(drop_these, axis=1, inplace=True)

#Rename columns
df_gg = df_gg.rename(columns = {"SERIES" : "Type", "Value": "Gender_gap", "Time": "Year"})

In [16]:
#Keep only the gender gap at the median 
I = df_gg.Type.str.contains('GWG5')
df_gg = df_gg.loc[I==True] 

# Reset old index
df_gg.reset_index(inplace = True, drop = True) # Drop old index too
df_gg.drop('Type', axis=1, inplace=True)

# Sort
df_gg = df_gg.sort_values(['Country','Year'])

# Reset old index
df_gg.reset_index(inplace = True, drop = True) # Drop old index too

In [17]:
# Country specific dataset
countries = ['Denmark', 'United States', 'Sweden','Germany','United Kingdom','OECD countries']
df_c_g = df_gg[df_gg['Country'].isin(countries)]
df_c_g = df_c_g.round(1)
# Reset old index
df_c_g.reset_index(inplace = True, drop = True) # Drop old index too
df_c_g

,Country,Year,Gender_gap
0,Denmark,2005,10.2
1,Denmark,2006,10.2
2,Denmark,2007,9.9
3,Denmark,2008,10.2
4,Denmark,2009,10.2
...,...,...,...
95,United States,2017,18.2
96,United States,2018,18.9
97,United States,2019,18.5
98,United States,2020,17.7


In [18]:
# Times series plot of the Gender Gap
import plotly.graph_objects as go

fig = go.Figure()

for country, df_c_g in df_c_g.groupby('Country'):
    if country == 'OECD countries':  # add condition to change line style for OECD countries
        fig.add_trace(go.Scatter(x=df_c_g['Year'], y=df_c_g['Gender_gap'], name=country, line=dict(color='black', dash='dash')))
    else:
        fig.add_trace(go.Scatter(x=df_c_g['Year'], y=df_c_g['Gender_gap'], name=country))

fig.update_layout(title='Development in Gender Gap over time',
                  xaxis_title='Year',
                  yaxis_title='Pct.')

fig.show()

The gender gap has generally been decreasing over the considered time period across all selected OECD-countries. On average, the gender gap has decreased from around 16 pct. in 2005 to approximately 12 pct. in 2021. The gender gap is generally lower in Denmark and Sweden and higher in United States and United Kingdom. In Germany, the gender gap is above the OECD-average over the entire period. 

In [19]:
# 2020 dataset for later use
df_gg_2020 = df_gg.loc[df_gg['Year']==2020]
df_gg_2020 = df_gg_2020.sort_values(['Country','Year'])
df_gg_2020.reset_index(inplace = True, drop = True) # Drop old index too

# c. Net child care costs
1) We read the OECD-data set for net child care costs across countries. We drop irrelevant columns and rename
2) We clean the data-set 
3) We make an interactive plot that illustrates the development in net child care costs over time for a selection of OECD-countries
4) We create a data set with only 2020-observations for later use

In our inagural project, we added a disutility of working in the labor market for women in order to make the model predictions fit data. We interpreted this as norms. According to Kleven et al. (2019), only female labor supply is affected by children, i.e., the child penalty only strikes women where having children is basically a non event for fathers. We therefore incorperated norms in the model based on an assumption that part of home production is related to child care. A lack of access to child care in society could potentially augment the child penalty as this could force women to stay at home when they have children (or at least work less). We investigate this hypothesis by plotting the net child care costs (as pct. of household income) across our selection of OECD-countries as we should expect child care costs to all things equal be higher when acess to formal child care is low. 

In [20]:
# Net child care costs 
# Drop collumns that we do not need
drop_these = ['LOCATION','Type of indicator','Net childcares cost by item','Family type','Earnings of the first adult','HBTOPUPS','TIME','Flag Codes','Flags']
df_nc.drop(drop_these, axis=1, inplace=True)

#Rename columns
df_nc = df_nc.rename(columns = {"TYPE" : "Type", "Value": "Cost", "Time": "Year"})

In [21]:
# Get one observation pr. year pr. country. 
# We consider percentage of net household income for two earner family with wages at the average. 
# We include social benefits and housing benefits
I = (df_nc['Type'] == 1) & (df_nc['FAMILY'] == '2EARNERC2C_67AW') & (df_nc['Include social assistance benefits'] == 'Yes') & (df_nc['Include housing benefits'] == 'Yes') & (df_nc['COMPONENTS'] == 5) & (df_nc['EARNINGS'] == '67AW') 

df_nc = df_nc.loc[I==True] 

# Reset old index
df_nc.reset_index(inplace = True, drop = True) # Drop old index too
df_nc.drop('Type', axis=1, inplace=True)
df_nc.drop('FAMILY', axis=1, inplace=True)

# Sort
df_nc = df_nc.sort_values(['Country','Year'])

# Reset old index
df_nc.reset_index(inplace = True, drop = True) # Drop old index too

In [22]:
# Keep only relevant countries
countries_2 = ['Denmark', 'United States', 'Sweden', 'Germany','United Kingdom','OECD - Total']
df_c_nc = df_nc[df_nc['Country'].isin(countries_2)]
df_c_nc = df_c_nc.round(1)

In [23]:
fig = go.Figure()

for country, df_c_nc in df_c_nc.groupby('Country'):
    if country == 'OECD - Total':  # add condition to change line style for OECD countries
        fig.add_trace(go.Scatter(x=df_c_nc['Year'], y=df_c_nc['Cost'], name=country, line=dict(color='black', dash='dash')))
    else:
        fig.add_trace(go.Scatter(x=df_c_nc['Year'], y=df_c_nc['Cost'], name=country))

fig.update_layout(title='Net costs for a household with two children using childcare',
                  xaxis_title='Year',
                  yaxis_title='Pct. of household income')

fig.show()

As expected, net costs for a household with two children using childcare are generally higher in the UK and the US where the gender gap is also higher. It is interesting that the costs in Denmark are generally higher than in Sweden and Germany when the gender gap is the smallest here. This may be due to data differences and how countries determine and calculate costs. 

# d. Parental leave, formal child care and gender gap

1) We read the OECD-data set for parental leave and formal child care across countries. We drop irrelevant columns and rename. We keep only observatins for 2020. 
2) We clean the two data-sets
3) We merge the two data-sets and add the gender gap data for 2020 as well 
4) We make an interactive plot that illustrates the development in net child care costs over time for a selection of OECD-countries
5) We create a bubble diagram that shows formal child care costs as pct. of GDP and number of weeks fathers can take parental leave. The bubble size indicates the size of the gender gap.


In [24]:
# Parental leave dataset 
drop_these = ['COU','Indicator','SEX','Sex','AGE','Age Group','TIME','Unit','Unit Code','PowerCode Code','PowerCode','Reference Period Code','Reference Period','Flag Codes','Flags']
df_pl.drop(drop_these, axis=1, inplace=True)

#Rename columns
df_pl = df_pl.rename(columns = {"IND" : "Type", "Value": "Father_leave", "Time": "Year"})

In [25]:
#Keep only EMP18_PAT variable
I = df_pl.Type.str.contains('EMP18_PAT')
df_pl = df_pl.loc[I==True] 

# Reset old index
df_pl.reset_index(inplace = True, drop = True) # Drop old index too
df_pl.drop('Type', axis=1, inplace=True)

# Sort
df_pl = df_pl.sort_values(['Country','Year'])

# Reset old index
df_pl.reset_index(inplace = True, drop = True) # Drop old index too

In [26]:
# 2020
df_pl_2020 = df_pl.loc[df_pl['Year']==2020]
df_pl_2020 = df_pl_2020.sort_values(['Country','Year'])
df_pl_2020.reset_index(inplace = True, drop = True) # Drop old index too

In [27]:
# Child benefits dataset
drop_these = ['COU','Indicator','SEX','Sex','YEAR','Unit','Unit Code','PowerCode Code','PowerCode','Reference Period Code','Reference Period','Flag Codes','Flags']
df_f.drop(drop_these, axis=1, inplace=True)

In [28]:
#Keep FAM13
I = df_f.IND.str.contains('FAM13')
df_fam13 = df_f.loc[I==True] 
df_fam13 = df_fam13.rename(columns = {"IND" : "Type", "Value": "Formal_child_care"})

# Reset old index
df_fam13.reset_index(inplace = True, drop = True) # Drop old index too
df_fam13.drop("Type", axis=1, inplace=True)

# Sort
df_fam13 = df_fam13.sort_values(['Country','Year'])

# Reset old index
df_fam13.reset_index(inplace = True, drop = True) # Drop old index too

In [29]:
# 2020
df_fam13_2020 = df_fam13.loc[df_fam13['Year']==2020]
df_fam13_2020 = df_fam13_2020.sort_values(['Country','Year'])
df_fam13_2020.reset_index(inplace = True, drop = True) # Drop old index too

In [30]:
# Merge data-set
df_merged = pd.merge(df_gg_2020, df_pl_2020, on=['Country','Year'], how='outer')
df_merged = pd.merge(df_merged, df_fam13_2020, on=['Country','Year'], how='outer')

In [31]:
# Drop observations where gender gap is NaN
df_merged = df_merged.round(1)
df_merged.fillna(0, inplace=True)
index_to_drop = df_merged[df_merged['Gender_gap'] == 0].index
df_merged = df_merged.drop(index=index_to_drop)

In [32]:
# Make bubble plot

highlighted_countries = ['Denmark', 'Sweden', 'Germany','United Kingdom','United States']

# Create a color map for the markers
marker_colors = df_merged['Country'].apply(lambda country: 'red' if country in highlighted_countries else 'blue')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_merged['Father_leave'],
    y=df_merged['Formal_child_care'],
    mode='markers',
    marker=dict(
        size=df_merged['Gender_gap'],
        sizemode='area',
        sizeref=0.1,
        sizemin=5,
        color=marker_colors,  # Set the marker color based on the color map
        opacity=0.7,  # Set the opacity to make the markers semi-transparent
        line=dict(width=0.5, color='white'),  # Add a white border around the markers
    ),
    text=df_merged['Country'],
))

# Set the plot layout
fig.update_layout(
    title='Paternal leave and formal child care by country (2020)',
    xaxis_title='Paternity leave (Weeks)',
    yaxis_title='Formal child care (Pct. of GDP)',
)

# Display the plot in an interactive window
fig.show()

It seems as thoug formal child care and paternity leave are important drivers for lowering the gender gap. However, some there are some wierd outliers  (France, Korea and Japan). E.g. in Korea the gender gap is large but formal child care costs are high and it seems as though fathers have good acess to paternity leave. 

Again, there may be a problem with the validity of OECD-data and how countries differ in their determination and calculations. 

# Conclusion